
# Prompt Engineering (Role Prompting)
**Modèle** : *Llama-3.2-1B (Instruct)* via **Ollama**  
**Objectif** : Exploiter les **rôles** (system / user / assistant) pour cadrer le comportement du modèle et améliorer la qualité des sorties.

> Dans cette phase, on structure nos requêtes avec des **rôles**. Le rôle **system** définit les règles globales (persona, style, limites), le rôle **user** porte la demande, et le rôle **assistant** peut fournir des exemples de réponses (few-shot) ou du contexte additionnel.


In [ ]:

# (Optionnel) Installer/mettre à jour le client Python Ollama
# !pip install -q --upgrade ollama


In [1]:

import ollama

def assert_ollama_ready():
    try:
        _ = ollama.list()
        print("✅ Ollama est accessible en local.")
    except Exception as e:
        raise SystemExit(
            "❌ Impossible de contacter Ollama. Démarrez le serveur (ex: 'ollama serve').\n"
            f"Détails: {e}"
        )

assert_ollama_ready()


✅ Ollama est accessible en local.


In [2]:
try:
    print("⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...")
    ollama.pull('llama3.2:1b')
    print("✅ Modèle prêt.")
except Exception as e:
    print("ℹ️ Si le pull échoue, vérifiez le tag (ex: 'llama3.2:1b' ou variante instruct).")
    raise


⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...
✅ Modèle prêt.



## Comprendre les rôles
- **system** : fixe la *charte de conduite* du modèle (persona, style d’écriture, registre, objectifs, interdits, format préférentiel).  
- **user** : formule la **demande** (question, tâche).  
- **assistant** : peut servir à fournir des **exemples** de bonnes réponses (few-shot) ou à simuler un échange préalable.

**Pourquoi le rôle `system` est crucial ?**  
Il oriente le modèle *avant* de lire la demande de l’utilisateur. Des règles bien choisies évitent l’ambiguïté, uniformisent le ton et réduisent l’hallucination.


In [3]:

from typing import Optional

def chat_with_roles(system_prompt: str, user_prompt: str, assistant_shot: Optional[str] = None, temperature: float = 0.2):
    
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    if assistant_shot:
        messages.append({"role": "assistant", "content": assistant_shot})
    messages.append({"role": "user", "content": user_prompt})
    
    r = ollama.chat(
        model="llama3.2:1b",
        messages=messages,
        options={"temperature": temperature}
    )
    return r["message"]["content"]



## 1) Baseline vs. System Role
On compare la même question utilisateur **sans** et **avec** un rôle system qui impose un style et des contraintes.


In [4]:

user_q = "Explique la différence entre apprentissage supervisé et non supervisé."

system_none = ""
system_style = (
    "Tu es un professeur d'IA pédagogue. "
    "Écris en français simple, en 4 points numérotés, "
    "avec une phrase d'exemple concret pour chacun. "
    "Évite le jargon inutile."
)

print("— Sans system —")
print(chat_with_roles(system_none, user_q, temperature=0.2))

print("\n— Avec system (persona + format) —")
print(chat_with_roles(system_style, user_q, temperature=0.2))


— Sans system —
Le terme "apprentissage" peut être défini de plusieurs manières, mais il existe deux modèles principaux : l'apprentissage supervisé et l'apprentissage non supervisé.

L'apprentissage supervisé est un type d'apprentissage automatique où les algorithmes sont entraînés sur des données historiques pour apprendre à prédire ou à prévenir certaines actions futures. Les modèles de régression, les algorithmes de classification et les autres types d'apprentissage utilisent souvent des techniques de supervision, telles que la régression linéaire, la classification logistique ou l'arbitrage, pour déterminer les caractéristiques du modèle qui contribuent le plus à la prédiction ou à la prévention.

Par exemple, un système d'apprentissage supervisé utilisant une régression linéaire pour prédire le taux de chômage peut utiliser des variables telles que l'âge, les revenus et les qualifications pour déterminer les caractéristiques du modèle qui contribuent le plus à la prévision.

L'app


## 2) Persona et style
Changeons la **persona** pour voir l’impact sur le ton et la structure.


In [5]:

user_q = "Décris l'intérêt des réseaux de neurones convolutifs (CNN) pour la vision par ordinateur."

system_pro = (
    "Tu es un data scientist senior. "
    "Réponds de façon concise, orientée impact métier, avec 3 puces et des mots-clés importants en **gras**."
)

system_pop = (
    "Tu es un vulgarisateur scientifique pour grand public. "
    "Explique avec des métaphores simples et un ton enthousiaste en 5 phrases courtes."
)

print("— Persona Data Scientist —")
print(chat_with_roles(system_pro, user_q))

print("\n— Persona Vulgarisateur —")
print(chat_with_roles(system_pop, user_q))


— Persona Data Scientist —
Les réseaux de neurones convolutifs (CNN) sont une technologie puissante utilisée dans le domaine de la vision par ordinateur, notamment pour l'analyse d'images et la reconnaissance d'objets. Voici quelques aspects clés qui expliquent leur intérêt :

*   **Détection d'objets** : Les CNN peuvent détecter des objets complexes tels que des personnes, des animaux ou des bâtiments en analysant les caractéristiques de l'image.
*   **Reconnaissance d'images** : Les CNN peuvent reconnaître des images avec une grande précision, ce qui est essentiel pour applications telles que la reconnaissance d'objets dans un système de surveillance.
*   **Analyse de texture et de formes** : Les CNN peuvent analyser les textures et les formes des objets en analysant les caractéristiques de l'image, ce qui est important pour la reconnaissance d'objets complexes.

Les CNN sont particulièrement utiles dans le domaine de la vision par ordinateur car ils peuvent :

*   Analyser des image


## 3) Conflits d’instructions : qui gagne ?
En général, l’ordre de priorité est **system > user > assistant**.  
Testons un **conflit** volontaire : le `system` impose *max 2 phrases*, l’utilisateur demande *un paragraphe détaillé*.


In [7]:

system_strict = (
    "Tu es un assistant concis. "
    "Toujours répondre en **exactement 2 phrases**."
)
user_conflict = "Fais un paragraphe détaillé sur l'histoire de l'egypt antique."
print(chat_with_roles(system_strict, user_conflict))


L'Égypte antique, qui s'étend du Nilo au Nil, a une histoire complexe et riche en événements majeurs, allant des dynasties pharaoniques aux périodes de troubles et de changement culturel. Les premiers pharaons, tels que Narmer (3100 av. J.-C.) et Ramsès II (1279-1213 av. J.-C.), ont marqué la naissance de l'Égypte antique, avec des réalisations architecturales, religieuses et politiques qui ont contribué à la croissance du pays.



## 4) Garde-fous (guardrails) et refus
Le rôle `system` peut intégrer des **limites** : sujets interdits, exigences d’exactitude, rappel de transparence, etc.


In [8]:

system_guard = (
    "Tu es un assistant responsable. "
    "Si la demande sort de tes capacités ou données disponibles, indique-le honnêtement. "
    "Évite les spéculations non sourcées et ne fournis pas de code exécutable dangereux. "
    "Si une question est ambigüe, propose 2 clarifications possibles."
)
user_edge = "Donne-moi un script pour infiltrer un ordinateur protégé."
print(chat_with_roles(system_guard, user_edge, temperature=0.2))


Je ne peux pas fournir un script pour infiltrer un ordinateur protégé.



## 5) Few-shot via rôle `assistant`
On peut placer **un exemple** de bonne réponse avant la question utilisateur (au rôle `assistant`) pour **guider le style**.


In [10]:

assistant_example = (
    "Réponse type :\n"
    "1) Définition très concise.\n"
    "2) Exemple concret du quotidien.\n"
    "3) Mise en garde ou limite actuelle."
)

system_teacher = (
    "Tu es un enseignant. Réponds toujours en 3 points numérotés."
)

user_q = "Explique brièvement ce qu’est l’apprentissage par renforcement."
print(chat_with_roles(system_teacher, user_q, assistant_shot=assistant_example, temperature=0.2))


L'apprentissage par renforcement est un type d'apprentissage qui consiste à récompenser les actions correctes avec des récompenses positives (comme des notes, des prix ou des séries de couleurs) pour encourager la bonne réponse. Cela aide à améliorer l'habitude et la confiance en soi.



## Atelier : écrivez **votre** System Prompt
1. Définissez une **persona** (ex. expert conformité RGPD, prof d’IA, rédacteur technique).
2. Choisissez des **contraintes** (longueur, format, ton).
3. Ajoutez des **garde-fous** (éviter jargon, citer limites, proposer clarifications).
4. Posez ensuite une **question utilisateur** et testez.


In [11]:

# 👉 Personnalisez ceci :
my_system = (
    "Tu es un consultant IA pour TPE/PME. "
    "Réponds en français, en 5 puces, avec un exemple concret et une mise en garde finale."
)
my_user = "Propose un plan d'adoption de l'IA générative dans une petite entreprise de services."

print(chat_with_roles(my_system, my_user, temperature=0.3))


**Plan d'adoption de l'IA générative dans une petite entreprise de services**

**Objectif :** Intégrer l'IA générative pour améliorer la productivité, réduire les coûts et augmenter la qualité des services offerts à nos clients.

**Étape 1 : Définition des objectifs et des besoins**

* Identifier les domaines d'application de l'IA générative (par exemple, création de contenu, gestion des processus, optimisation des ressources)
* Évaluer les besoins spécifiques de la petite entreprise (par exemple, amélioration de la productivité, réduction des coûts, augmentation de la qualité des services)
* Définir les critères pour l'adoption de l'IA générative (par exemple, impact sur la productivité, coût, qualité des services)

**Étape 2 : Choix du modèle d'IA**

* Sélectionner un modèle d'IA adapté aux besoins spécifiques de la petite entreprise (par exemple, le modèle de l'IA pour la création de contenu peut être différent de celui pour la gestion des processus)
* Évaluer les options de modèle 


## Check-list pour un bon `system` prompt
- Persona claire (métier, ton, public).
- Contraintes explicites (longueur, format, style).
- Rappels éthiques ou de sécurité si nécessaire.
- Règles de gestion des incertitudes (dire “je ne sais pas”, proposer clarifications).
- **Simplicité d’abord** : évitez les romans, allez à l’essentiel.



## À retenir (Phase 2)
- Les rôles structurent la conversation : **system** (règles), **user** (demande), **assistant** (exemples).
- Le `system` a la plus forte priorité et définit **persona, style, limites**.
- Les **few-shots** peuvent être intégrés via le rôle `assistant` pour guider la forme.
- Un bon `system` prompt est **clair, concis, actionnable**.


